# Setup TDA API access

In [ ]:
import json
import requests
import datetime

In [ ]:
def load_config(filename, key):
    with open(filename, 'r') as f:
        return json.load(f)[key]
        
def update_refresh_token(refresh_file, config_file):
    token = load_config(refresh_file, 'refresh_token')
    client_id = load_config(config_file, 'client_id')
    endpoint = load_config(config_file, 'endpoint_auth')

    parameters = {
        'grant_type': 'refresh_token',
        'refresh_token': token,
        'access_type': 'offline',
        'code': '',
        'client_id': client_id
    }
    response = requests.post(url=endpoint, data=parameters).json()
    response['expiration_time'] = (datetime.datetime.now()+datetime.timedelta(seconds=response['refresh_token_expires_in'])).strftime('%Y%m%d_%H:%M:%S')
    with open('./Data/refresh_token.json', 'w') as f:
        json.dump(response, f)

def update_access_token(refresh_file, config_file):
    token = load_config(refresh_file, 'refresh_token')
    client_id = load_config(config_file, 'client_id')
    endpoint = load_config(config_file, 'endpoint_auth')
    
    parameters = {
        'grant_type': r'refresh_token',
        'refresh_token': token,
        'access_type': '',
        'code': '',
        'client_id': client_id
    }
    response = requests.post(url=endpoint, data=parameters).json()
    response['expiration_time'] = (datetime.datetime.now()+datetime.timedelta(seconds=response['expires_in'])).strftime('%Y%m%d_%H:%M:%S')
    with open('./Data/access_token.json', 'w') as f:
        json.dump(response, f)

def update_token(refresh_file, access_file, config_file):
    refresh_expiration = load_config(refresh_file, 'expiration_time')
    access_expiration = load_config(access_file, 'expiration_time')
    refresh_expires = datetime.datetime.strptime(refresh_expiration, '%Y%m%d_%H:%M:%S')-datetime.datetime.now()
    access_expires = datetime.datetime.strptime(access_expiration, '%Y%m%d_%H:%M:%S')-datetime.datetime.now()
    
    if refresh_expires < datetime.timedelta(seconds=604800):
        update_refresh_token(refresh_file, config_file)
        print('Refresh token updated.')
    if access_expires < datetime.timedelta(seconds=360):   
        update_access_token(refresh_file, config_file)
        print('Access token updated.')
    else:
        print("Tokens are still active. Refresh expires in {}. Access expires in: {}".format(refresh_expires, access_expires))

In [ ]:
config_file = './Data/config.json'
refresh_file = './Data/refresh_token.json'
access_file = './Data/access_token.json'

In [ ]:
update_token(refresh_file, access_file, config_file)

# Query using TDA API

In [ ]:
def get_account(refresh_file, access_file, config_file, save=0):
    update_token(refresh_file, access_file, config_file)
    access_token = load_config(access_file, key='access_token')
    account_number = load_config(config_file, key='account_number')
    endpoint = load_config(config_file, key='endpoint_account').format(account_number)
    payload = {
        'Authorization': r'Bearer {}'.format(access_token)
    }
    response = requests.get(url=endpoint, headers=payload).json()
    if save:
        time_stamp = str(datetime.date.today())
        with open('./Data/account_summary_{}.json'.format(time_stamp), 'w') as account_data:
            json.dump(response, account_data)
    return response

In [ ]:
get_account(refresh_file, access_file, config_file, save=1)

In [ ]:
def get_transactions(refresh_file, access_file, config_file, start_date, end_date, save=0):
    """
    format: <yyyy-MM-dd>
    """
    update_token(refresh_file, access_file, config_file)
    access_token = load_config(access_file, key='access_token')
    account_number = load_config(config_file, key='account_number')
    endpoint = load_config(config_file, key='endpoint_transaction').format(account_number)

    payload = {
               'Authorization': r'Bearer {}'.format(access_token)
    }
    parameters = {
        'type': 'ALL',
        'startDate': start_date,
        'endDate': end_date
    }
    response = requests.get(url=endpoint, headers=payload, params=parameters).json()
    if save:
        with open('./Data/{}_{}_transactions.json'.format(start_date, end_date), 'w') as transaction_data:
            json.dump(response, transaction_data)
    return response 

In [ ]:
get_transactions(refresh_file, access_file, config_file, '2021-01-01', '2021-05-01', save=1)

# Place Order Example

In [ ]:
def create_order(category='stock', strategy='buy'):
    if category=='stock':
        order = {
            "orderType": "MARKET",
            "session": "NORMAL",
            "duration": "DAY",
            "orderStrategyType": "SINGLE",
            "orderLegCollection": [
                {
                    "instruction": "Buy",
                    "quantity": 15,
                    "instrument": {
                        "symbol": "AMD",
                        "assetType": "EQUITY"
                    }
                }
            ]
        }
    elif category=='option':
        order = {
            "complexOrderStrategyType": "NONE",
            "orderType": "LIMIT",
            "session": "NORMAL",
            "price": "1.45",
            "duration": "DAY",
            "orderStrategyType": "SINGLE",
            "orderLegCollection": [
                {
                    "instruction": "SELL_TO_OPEN",
                    "quantity": 1,
                    "instrument": {
                        "symbol": "PLTR_061821P22",
                        "assetType": "OPTION"
                    }
                }
            ]
        }
    return order 

def place_order(refresh_file, access_file, config_file, order):
    update_token(refresh_file, access_file, config_file)
    access_token = load_config(access_file, key='access_token')
    account_number = load_config(config_file, key='account_number')

    endpoint = load_config(config_file, key='endpoint_orders').format(account_number)
    payload = {'Authorization': r'Bearer {}'.format(access_token)}
    
    parameters = order
    
    response = requests.post(url=endpoint, headers=payload, json=parameters)
    return response

In [ ]:
order = create_order('option')

In [ ]:
place_order(refresh_file, access_file, config_file, order)

# Cancel Order by Order ID

In [ ]:
def get_order_id(refresh_file, access_file, config_file, order_date, save=0):
    """
    format: <yyyy-MM-dd>
    """
    update_token(refresh_file, access_file, config_file)
    access_token = load_config(access_file, key='access_token')
    account_number = load_config(config_file, key='account_number')
    endpoint = load_config(config_file, key='endpoint_orders').format(account_number)

    payload = {
               'Authorization': r'Bearer {}'.format(access_token)
    }
    parameters = {
        "maxResults": 1,
        "fromEnteredTime": order_date,
        "toEnteredTime": order_date,
        "status": "WORKING"
    }
    response = requests.get(url=endpoint, headers=payload, params=parameters).json()[0]
    if save:
        with open('./Data/working_orders.json'.format(order_date), 'w') as working_data:
            json.dump(response, working_data)
    return response['orderId']

def cancel_order(refresh_file, access_file, config_file, order_id):
    update_token(refresh_file, access_file, config_file)
    access_token = load_config(access_file, key='access_token')
    account_number = load_config(config_file, key='account_number')
    endpoint = load_config(config_file, key='endpoint_cancel_order').format(account_number, order_id)

    payload = {'Authorization': r'Bearer {}'.format(access_token)}
    
    response = requests.delete(url=endpoint, headers=payload)
    return response


In [ ]:
order_id = get_order_id(refresh_file, access_file, config_file, '2021-05-02')
cancel_order(refresh_file, access_file, config_file, order_id)